# Creación de una Malla 2D con Gmsh

Este notebook demuestra cómo utilizar la API de Python de **Gmsh** para crear una malla de elementos finitos para un problema de tensión plana: una placa rectangular con un agujero en el centro.

El proceso se divide en:
1.  Definir la geometría (puntos, líneas, superficies).
2.  Asignar "Grupos Físicos" a las entidades geométricas para poder aplicar cargas y condiciones de frontera más adelante.
3.  Generar la malla 2D.
4.  Guardar la malla en un archivo `.msh` para ser utilizado por el solver.

In [1]:
import gmsh

### Paso 1: Inicialización y Parámetros

In [2]:
# Inicializamos Gmsh y creamos un nuevo modelo.
gmsh.initialize()
gmsh.model.add("placa_con_agujero")

### Paso 2: Definición de la Geometría

In [3]:
# Definimos los tamaños de malla. Usaremos una malla más fina (`tm_refinado`)
# alrededor del agujero para capturar mejor la concentración de esfuerzos.
tm_general = 2E-2
tm_refinado = 2E-3

In [4]:
# Puntos para el rectángulo exterior
p = gmsh.model.geo.addPoint(0,   0,   0, tm_general, 1) 
p = gmsh.model.geo.addPoint(0.4, 0,   0, tm_general, 2) 
p = gmsh.model.geo.addPoint(0.4, 0.2, 0, tm_general, 3)
p = gmsh.model.geo.addPoint(0,   0.2, 0, tm_general, 4)

In [5]:
# Puntos para el círculo interior (agujero)
radio = 3E-2
centro_x, centro_y = 0.2, 0.1
p = gmsh.model.geo.addPoint(centro_x,          centro_y,          0, tm_general, 5) # Punto central
p = gmsh.model.geo.addPoint(centro_x + radio,  centro_y,          0, tm_refinado, 6) # Punto derecho del círculo
p = gmsh.model.geo.addPoint(centro_x - radio,  centro_y,          0, tm_refinado, 7) # Punto izquierdo del círculo

In [6]:
# Creamos las líneas del rectángulo exterior.
l = gmsh.model.geo.addLine(1, 2, 1)
l = gmsh.model.geo.addLine(2, 3, 2)
l = gmsh.model.geo.addLine(3, 4, 3)
l = gmsh.model.geo.addLine(4, 1, 4)

In [7]:
# Creamos los arcos para el círculo.
arc = gmsh.model.geo.addCircleArc(6, 5, 7, 5)  # Arco superior
arc = gmsh.model.geo.addCircleArc(7, 5, 6, 6)  # Arco inferior

In [8]:
# Creamos los "Curve Loops" que definen los contornos cerrados.
loop_exterior = gmsh.model.geo.addCurveLoop([1, 2, 3, 4], 1)
loop_interior = gmsh.model.geo.addCurveLoop([5, 6], 2)

In [9]:
# Creamos la superficie plana, especificando que tiene un agujero.
s = gmsh.model.geo.addPlaneSurface([loop_exterior, loop_interior], 1)

### Paso 3: Definición de Grupos Físicos

Los grupos físicos son esenciales. Asignan etiquetas a las entidades geométricas, que luego son leídas por el solver para aplicar condiciones de contorno.

In [10]:
# Grupo para la superficie 2D (donde se crearán los elementos).
gf = gmsh.model.geo.addPhysicalGroup(2, [1], 1, "superficie") 

In [11]:
# Grupos para las líneas (bordes) 1D.
gf = gmsh.model.geo.addPhysicalGroup(1, [2], 101, "lado_cargas") # Borde derecho
gf = gmsh.model.geo.addPhysicalGroup(1, [4], 102, "lado_fijo")   # Borde izquierdo

### Paso 4: Generación de la Malla y Visualización

In [12]:
# Sincronizamos el modelo CAD con el modelo de Gmsh.
gmsh.model.geo.synchronize()

In [13]:
# Generamos la malla 2D.
gmsh.model.mesh.generate(2)
gmsh.option.setNumber('Mesh.SurfaceFaces', 1)
gmsh.option.setNumber('Mesh.Points', 1)   

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 60%] Meshing curve 4 (Line)
Info    : [ 70%] Meshing curve 5 (Circle)
Info    : [ 90%] Meshing curve 6 (Circle)
Info    : Done meshing 1D (Wall 0.000377951s, CPU 0.000596s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0154483s, CPU 0.015052s)
Info    : 905 nodes 1815 elements


In [14]:
# Guardamos la malla en un archivo.
nombre_archivo = 'ejem_1.msh'
gmsh.write(nombre_archivo)

Info    : Writing 'ejem_1.msh'...
Info    : Done writing 'ejem_1.msh'


In [15]:
# Opcional: Abrimos la GUI de Gmsh para inspeccionar la malla visualmente.
# Se puede cerrar la ventana para que el script continúe.
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.14.0
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20250703
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib tinyobjloader
FLTK version  : 1.3.11
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.8.1
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


X_ChangeProperty: BadValue (integer parameter out of range for operation) 0x0


In [16]:
# Finalizamos la sesión de Gmsh.
gmsh.finalize()